
# ETL Silver -> Gold (Camada DW)

O objetivo deste ETL é transformar os dados analíticos da **camada Silver** em uma estrutura dimensional **(modelo estrela)** pronta para consultas no **Metabase**, permitindo análises de mercado de criptomoedas com métricas consolidadas e dimensões otimizadas.

## 1. Extração
Os dados são extraídos da camada **Silver**, principalmente da tabela currencies_data

In [1]:

#!/usr/bin/env py
import pandas as pd
from sqlalchemy import create_engine, text
import datetime
from dotenv import load_dotenv
import os
from pathlib import Path

load_dotenv('../.env')

DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '5432')
POSTGRES_DB = os.getenv('POSTGRES_DB', 'crypto_db')
POSTGRES_USER = os.getenv('POSTGRES_USER', 'postgres')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD', 'postgres')

# Primeiramente vamos estabelecer conexão com o banco de dados e criar o schema **dw** .
connection_string = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{DB_HOST}:{DB_PORT}/{POSTGRES_DB}"
engine = create_engine(connection_string)
ddl_path = Path('../data/gold/ddl_gold.sql')

with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS dw.fact_market CASCADE;"))
    conn.execute(text("DROP TABLE IF EXISTS dw.dim_currency CASCADE;"))
    conn.execute(text("DROP TABLE IF EXISTS dw.dim_time CASCADE;"))
    conn.commit()
    
    with open(ddl_path, 'r') as file:
        sql_script = file.read()
        conn.execute(text(sql_script))
    conn.commit()
print("Conectado ao banco e o DDL feito.")

Conectado ao banco e o DDL feito.


Realizar de seguida a leitura dos dados da camada Silver

In [2]:
query_silver = "SELECT * FROM currencies_data;"
silver_df = pd.read_sql(query_silver, engine)
print("Análise dos dados Silver:")
print(f"Total registros: {len(silver_df)}")
print(f"Símbolos únicos: {silver_df['symbol'].nunique()}")
print(f"Datas únicas: {silver_df['last_updated'].nunique()}")
print(f"Primeiras linhas:\n{silver_df[['symbol', 'name', 'last_updated']].head(10)}")

Análise dos dados Silver:
Total registros: 141575
Símbolos únicos: 1
Datas únicas: 4
Primeiras linhas:
  symbol         name            last_updated
0    USD      Bitcoin  2023-09-04 14:57:00+00
1    USD     Ethereum  2023-09-04 14:57:00+00
2    USD  Tether USDt  2023-09-04 14:57:00+00
3    USD          BNB  2023-09-04 14:57:00+00
4    USD          XRP  2023-09-04 14:57:00+00
5    USD     USD Coin  2023-09-04 14:57:00+00
6    USD      Cardano  2023-09-04 14:57:00+00
7    USD     Dogecoin  2023-09-04 14:57:00+00
8    USD       Solana  2023-09-04 14:57:00+00
9    USD         TRON  2023-09-04 14:57:00+00


## Problemática
Um dos problemas que temos é a duplicação de dados, e precisamos resolver isto para manter a consistência dos dados

In [9]:
# Estratégia: Criar uma chave única combinando name + cmc_rank
silver_df['unique_currency_id'] = silver_df['name'] + '_' + silver_df['cmc_rank'].astype(str)

print(f"Moedas únicas antes: {silver_df['name'].nunique()}")
print(f"Moedas únicas após (name + cmc_rank): {silver_df['unique_currency_id'].nunique()}")

# Verificar o caso do "X"
x_currencies = silver_df[silver_df['name'] == 'X'][['name', 'cmc_rank', 'unique_currency_id']].drop_duplicates()
print(f"\nMoedas chamadas 'X':")
print(x_currencies)

# AGORA remover duplicatas usando a chave única corrigida
initial_count = len(silver_df)

silver_df_clean = silver_df.sort_values('market_cap', ascending=False).drop_duplicates(
    subset=['unique_currency_id', 'last_updated'], 
    keep='first'
)

print(f"\n--- RESULTADOS DA LIMPEZA ---")
print(f"Registros antes: {initial_count}")
print(f"Registros após: {len(silver_df_clean)}")
print(f"Duplicatas removidas: {initial_count - len(silver_df_clean)}")

# Validação
remaining_duplicates = silver_df_clean.duplicated(subset=['unique_currency_id', 'last_updated']).sum()
print(f"Duplicatas restantes: {remaining_duplicates}")

silver_df_clean = silver_df.sort_values(['market_cap', 'cmc_rank'], ascending=[False, True])
silver_df_clean = silver_df_clean.drop_duplicates(subset=['name', 'last_updated'], keep='first')
silver_df = silver_df_clean

Moedas únicas antes: 9193
Moedas únicas após (name + cmc_rank): 10847

Moedas chamadas 'X':
     name  cmc_rank unique_currency_id
2357    X      2257             X_2257

--- RESULTADOS DA LIMPEZA ---
Registros antes: 20068
Registros após: 20068
Duplicatas removidas: 0
Duplicatas restantes: 0


## Transformação

### Criação das tabelas Dimensões

Três tabelas dimensionais são criadas para normalizar e estruturar os dados:

* **`dim_currency`**

  * Contém informações sobre as moedas digitais.
  * Colunas principais:

    * `SK_Currency` → Surrogate Key (PK)
    * `NK_Symbol` → Identificador natural (ex.: BTC, ETH)
    * `Name`, `CMC_Rank`, `Is_Active`

* **`dim_time`**

  * Representa as dimensões temporais (datas de atualização).
  * Colunas principais:

    * `SK_Time` → Surrogate Key (PK)
    * `NK_Date` → Data de referência (last_updated)
    * `Year`, `Month`, `Day`, `Hour`

Essas dimensões são carregadas no schema `dw` com o método `to_sql(if_exists="replace")`, garantindo consistência a cada execução.

### Criação da tabela Fato **fact_market**

A tabela fato integra as dimensões com as métricas numéricas da Silver.
O processo é feito em etapas:

1. **Merge com `dim_currency`** -> Relaciona moedas via `symbol`.
2. **Merge com `dim_time`** -> Relaciona datas via `last_updated`.
<!-- 3. **Merge com `dim_market_status`** -> Relaciona indicadores de mercado. -->

Após as junções, são selecionadas as medidas relevantes:

| Tipo    | Coluna                     | Descrição                       |
| ------- | -------------------------- | ------------------------------- |
| FK      | `FK_SK_Currency`           | Referência à moeda              |
| FK      | `FK_SK_Time`               | Referência ao tempo             |
| FK      | `FK_SK_Market_Status`      | Referência ao status de mercado |
| Métrica | `Price_Value`              | Preço atual                     |
| Métrica | `Volume_24h_Amount`        | Volume negociado em 24h         |
| Métrica | `Market_Cap_Value`         | Valor de mercado                |
| Métrica | `Percent_Change_1h_Value`  | Variação percentual em 1h       |
| Métrica | `Percent_Change_24h_Value` | Variação percentual em 24h      |
| Métrica | `Percent_Change_7d_Value`  | Variação percentual em 7 dias   |

Por fim, é criada uma chave substituta (`SK_Fact_Market`) que identifica unicamente cada registro na tabela fato.

In [4]:
dim_currency = (
    silver_df.sort_values('market_cap', ascending=False)
    .groupby('name')
    .first()
    .reset_index()[['name', 'cmc_rank', 'date_added', 'is_active']]
    .sort_values('cmc_rank')
    .reset_index(drop=True)
    .rename(columns={
        "name": "NK_Name",
        "cmc_rank": "CMC_Rank", 
        "date_added": "Date_Added",
        "is_active": "Is_Active",
    })
)

dim_currency["SK_Currency"] = range(1, len(dim_currency) + 1)
print(f"Dim_Currency: {len(dim_currency)} moedas")
print(f"Exemplo: {dim_currency[['NK_Name', 'CMC_Rank']].head(10)}")

# Adicionar surrogate key
dim_currency["SK_Currency"] = range(1, len(dim_currency) + 1)

print(f"Dim_Currency: {len(dim_currency)} moedas únicas")

main_coins = ['Bitcoin', 'Ethereum', 'Tether USDt', 'BNB', 'XRP', 'USD Coin']

# Adiciona SK_Currency
dim_currency["SK_Currency"] = range(1, len(dim_currency) + 1)
print(f"Dim_Currency criada com {len(dim_currency)} registros.")

# Dim_Time
# Extrai datas de last_updated
silver_df["last_updated"] = pd.to_datetime(silver_df["last_updated"], errors="coerce")
silver_df["last_updated"] = pd.to_datetime(silver_df["last_updated"], errors="coerce")
dim_time = (
    silver_df[["last_updated"]]
    .dropna()
    .drop_duplicates()
    .sort_values("last_updated")
    .reset_index(drop=True)
    .rename(columns={"last_updated": "NK_Date"})
)

# Extrair atributos temporais
dim_time["Year_Number"] = dim_time["NK_Date"].dt.year
dim_time["Month_Number"] = dim_time["NK_Date"].dt.month
dim_time["Day_Number"] = dim_time["NK_Date"].dt.day
dim_time["Hour_Number"] = dim_time["NK_Date"].dt.hour
dim_time["Day_Of_Week"] = dim_time["NK_Date"].dt.day_name()
dim_time["Month_Name"] = dim_time["NK_Date"].dt.month_name()
dim_time["Quarter_Number"] = dim_time["NK_Date"].dt.quarter
dim_time["Is_Weekend"] = dim_time["NK_Date"].dt.dayofweek >= 5

dim_time["SK_Time"] = range(1, len(dim_time) + 1)
print(f"Dim_Time criada com {len(dim_time)} timestamps únicos.")

Dim_Currency: 9193 moedas
Exemplo:        NK_Name  CMC_Rank
0      Bitcoin         1
1     Ethereum         2
2  Tether USDt         3
3          BNB         4
4          XRP         5
5     USD Coin         6
6      Cardano         7
7     Dogecoin         8
8       Solana         9
9         TRON        10
Dim_Currency: 9193 moedas únicas
Dim_Currency criada com 9193 registros.
Dim_Time criada com 4 timestamps únicos.


De seguida, vai ser feita a população das tabelas Dimensões no schema dw. Mas antes é necessário dropar as tabelas com segurança.

In [5]:
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS dw.fact_market CASCADE;"))
    conn.execute(text("DROP TABLE IF EXISTS dw.dim_currency CASCADE;"))
    conn.execute(text("DROP TABLE IF EXISTS dw.dim_time CASCADE;"))
    conn.commit()
    
dim_currency.to_sql("dim_currency", engine, schema="dw", if_exists="replace", index=False)
dim_time.to_sql("dim_time", engine, schema="dw", if_exists="replace", index=False)
print("Dimensões carregadas no schema dw.")

Dimensões carregadas no schema dw.


Criação da Tabela de Fatos (Fact_Market)

In [10]:

fact_market = silver_df.merge(
    dim_currency, 
    left_on="name",
    right_on="NK_Name", 
    how="left"  # Manter TODOS os registros silver
)
print(f"Após merge currency: {len(fact_market)} registros")

fact_market = fact_market.merge(
    dim_time, 
    left_on="last_updated", 
    right_on="NK_Date", 
    how="left"  # Manter TODOS os registros
)
print(f"Após merge time: {len(fact_market)} registros")

print(f"\n=== VERIFICAÇÃO DOS MERGES ===")
print(f"Registros sem currency: {fact_market['SK_Currency'].isna().sum()}")
print(f"Registros sem time: {fact_market['SK_Time'].isna().sum()}")

fact_market_final = fact_market[
    [
        "SK_Currency",
        "SK_Time",
        "price",
        "volume_24h", 
        "market_cap",
        "percent_change_1h",
        "percent_change_24h",
        "percent_change_7d",
        "dominance",
        "fully_dillutted_market_cap",
        "market_cap_by_total_supply",
        "ytd_price_change_percentage"
    ]
].rename(columns={
    "SK_Currency": "FK_SK_Currency",
    "SK_Time": "FK_SK_Time", 
    "price": "Price_Value",
    "volume_24h": "Volume_24h_Amount",
    "market_cap": "Market_Cap_Value",
    "percent_change_1h": "Percent_Change_1h_Value",
    "percent_change_24h": "Percent_Change_24h_Value",
    "percent_change_7d": "Percent_Change_7d_Value",
    "dominance": "Dominance_Value",
    "fully_dillutted_market_cap": "Fully_Diluted_Market_Cap_Value",
    "market_cap_by_total_supply": "Market_Cap_By_Total_Supply_Value",
    "ytd_price_change_percentage": "YTD_Price_Change_Percentage_Value"
})

fact_market_final["SK_Fact_Market"] = range(1, len(fact_market_final) + 1)

print(f"Dimensões da fact_market após seleção: {fact_market.shape}")

expected_records = len(dim_currency) * len(dim_time) 
actual_records = len(fact_market)

print(f"\n=== RESULTADO FINAL ===")
print(f"Registros silver: {len(silver_df)}")
print(f"Registros fact_market: {len(fact_market_final)}")
print(f"Moedas únicas: {len(dim_currency)}")
print(f"Timestamps únicos: {len(dim_time)}")

if len(silver_df) == len(fact_market_final):
    print("SUCESSO! Modelo dimensional consistente!")
else:
    print(f"PROBLEMA: {len(silver_df) - len(fact_market_final)} registros faltando")

print(f"\n=== CARREGANDO NO BANCO ===")
fact_market_final.to_sql("fact_market", engine, schema="dw", if_exists="replace", index=False)
print(f"Fact_Market salva com {len(fact_market_final)} registros")

# Adicionar chave primária e salvar
fact_market["SK_Fact_Market"] = range(1, len(fact_market) + 1)

print(f"\n=== SALVANDO NO BANCO ===")
print(f"Colunas finais: {fact_market.columns.tolist()}")

fact_market.to_sql("fact_market", engine, schema="dw", if_exists="replace", index=False)
print(f"Fact_Market criada com {len(fact_market)} registros.")

Após merge currency: 20068 registros
Após merge time: 20068 registros

=== VERIFICAÇÃO DOS MERGES ===
Registros sem currency: 0
Registros sem time: 0
Dimensões da fact_market após seleção: (20068, 39)

=== RESULTADO FINAL ===
Registros silver: 20068
Registros fact_market: 20068
Moedas únicas: 9193
Timestamps únicos: 4
SUCESSO! Modelo dimensional consistente!

=== CARREGANDO NO BANCO ===
Fact_Market salva com 20068 registros

=== SALVANDO NO BANCO ===
Colunas finais: ['cmc_rank', 'name', 'symbol', 'market_pair_count', 'circulating_supply', 'total_supply', 'max_supply', 'is_active', 'last_updated', 'date_added', 'price', 'volume_24h', 'market_cap', 'percent_change_1h', 'percent_change_24h', 'percent_change_7d', 'percent_change_30d', 'percent_change_60d', 'percent_change_90d', 'fully_dillutted_market_cap', 'market_cap_by_total_supply', 'dominance', 'ytd_price_change_percentage', 'unique_currency_id', 'NK_Name', 'CMC_Rank', 'Date_Added', 'Is_Active', 'SK_Currency', 'NK_Date', 'Year_Number'

### Carga
Após o tratamento e junção das dimensões:

* Cada dimensão é carregada no schema `dw` (`dim_currency`, `dim_time`);
* A tabela fato `fact_market` é carregada separadamente, **sem recriar as dimensões**, evitando erros de dependência (FK);
* O Metabase se conecta diretamente ao schema `dw`, tornando as dimensões e fatos disponíveis para dashboards e análises exploratórias.

Este passo é opcional, porém importante para a verificação de integridade dos dados. Em um processo de ETL como o da camada Silver -> Gold, a verificação de integridade dos dados é fundamental para garantir que o Data Warehouse (DW) reflita informações consistentes, confiáveis e relacionáveis.

No contexto deste projeto que envolve métricas financeiras de criptomoedas isso é ainda mais crítico, porque qualquer valor incorreto ou relacionamento quebrado pode distorcer análises e dashboards.

In [7]:
with engine.connect() as conn:
    count_fact = conn.execute(text("SELECT COUNT(*) FROM dw.fact_market;")).scalar()
    count_currency = conn.execute(text("SELECT COUNT(*) FROM dw.dim_currency;")).scalar()
    print(f"Registros: {count_fact} fatos, {count_currency} moedas únicas.")

Registros: 20068 fatos, 9193 moedas únicas.


In [12]:
with engine.connect() as conn:
    count_fact = conn.execute(text("SELECT COUNT(*) FROM dw.fact_market;")).scalar()
    count_currency = conn.execute(text("SELECT COUNT(*) FROM dw.dim_currency;")).scalar()
    
    print(f"\nVERIFICAÇÃO BANCO:")
    print(f"   Fact_Market: {count_fact} registros")
    print(f"   Dim_Currency: {count_currency} moedas")
    
    # Verificar se as moedas principais estão presentes
    for coin in main_coins:
        result = conn.execute(text(f"SELECT COUNT(*) FROM dw.dim_currency WHERE \"NK_Name\" = '{coin}';")).scalar()
        print(f"   {coin}: {'OK' if result > 0 else 'FALTANDO'}") 


VERIFICAÇÃO BANCO:
   Fact_Market: 20068 registros
   Dim_Currency: 9193 moedas
   Bitcoin: OK
   Ethereum: OK
   Tether USDt: OK
   BNB: OK
   XRP: OK
   USD Coin: OK
